In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from functools import partial

from IPython.display import Markdown as md, HTML
from nycschools import schools, geo, ui, class_size


References
-----------

Allen, R., & Vignoles, A. (2007). What should an index of school segregation measure? _Oxford Review of Education_, _33_(5), 643–668. https://doi.org/10.1080/03054980701366306

Frankel, D. M., & Volij, O. (2011). Measuring school segregation. _Journal of Economic Theory_, _146_(1), 1–38. https://doi.org/10.1016/j.jet.2010.10.008

Reardon, S. F., & O’Sullivan, D. (2004). Measures of Spatial Segregation. _Sociological Methodology_, _34_(1), 121–162. https://doi.org/10.1111/j.0081-1750.2004.00150.x

Sakoda, J. M. (1981). A Generalized Index of Dissimilarity. _Demography_, _18_(2), 245–250. https://doi.org/10.2307/2061096





In [2]:
# compare function to: 
# https://coascenters.howard.edu/dissimilarity-index-tutorial
test_data = pd.DataFrame(columns=["tract","b","w"], data=[[1,50,10],[2,200,40],[3,10,100],[4,30,200],[5,10,150]])
display(test_data)
B = test_data["b"].sum()
W = test_data["w"].sum()
assert (B, W) == (300, 500)
test_data["d_b"] = test_data["b"]/B
test_data["d_w"] = test_data["w"]/W
D = (test_data["d_b"] - test_data["d_w"]).abs().sum() / 2
assert abs(D - 0.73) < 0.01

D

,tract,b,w
0,1,50,10
1,2,200,40
2,3,10,100
3,4,30,200
4,5,10,150


0.7333333333333334

In [8]:
from nycschools import segregation as seg
long = pd.DataFrame(columns=["u","b","w"], data=[[1,50,10],[2,200,40],[3,10,100],[4,30,200],[5,10,150]])
long = long.melt(id_vars="u", value_vars=["b","w"], var_name="g", value_name="n").sort_values("u")
D = seg.dissimilarity(long, "w")
assert abs(D - 0.73) < .01

0.7333333333333334

In [4]:
df = schools.load_school_demographics()
# create some new columns
df["black_hispanic_n"] = df["black_n"] + df["hispanic_n"]
df["non_white_n"] = df["total_enrollment"] + df["white_n"]
df["asian_white_n"] = df["asian_n"] + df["white_n"]


In [5]:
def calc_D(df, group_col, n_col):
    """Calculate D for a single group vs. the rest of the population"""
    data = df.copy()
    N_i = data[n_col] - data[group_col] # non-group total for each location i
    G = data[group_col].sum() # group total
    N = N_i.sum()      # non-group total
    G_prop = data[group_col]/G
    N_prop = N_i/N
    D = (G_prop - N_prop).abs().sum() / 2
    return D

# check our pair-wise function with the test data
test_data["pop"] = test_data["b"] + test_data["w"]
D = calc_D(test_data, "b", "pop")
assert abs(D - 0.73) < 0.01
round(D, 4)

0.7333

In [7]:
def calc_GDI(data):
    data = data.copy()
    groups = ["asian", "black", "hispanic", "white"]
    data["pop"] = data[[g+"_n" for g in groups]].sum(axis=1)
    D_data = pd.DataFrame(columns=["group", "D"], data=[[g, calc_D(data, g+"_n", "pop")] for g in groups])
    # D_data["group"] = D_data.group.apply(lambda x: "D_" + group)
    weights = D_data.group.apply(lambda x: data[x+"_pct"].mean())
    # calculate a weighted mean of D_data.D with numpy
    GDI = np.average(D_data.D, weights=weights)
    D_data = pd.concat([D_data, pd.DataFrame({"group":"GDI", "D":GDI}, index=[0])])
    D_data =  D_data.set_index("group").transpose()
    data.columns.name = None
    D_data.columns = ["D_" + c for c in D_data.columns][:-1] + ["GDI"]
    return D_data.reset_index(drop=True)

x = calc_GDI(df[(df.ay == df.ay.max()) & (df.district == 13)])
x

,D_asian,D_black,D_hispanic,D_white,GDI
0,0.684352,0.510815,0.312506,0.403005,0.456955


In [33]:
def calc_D(data):
    # Get the totals
    A = data['asian_n'].sum()
    B = data['black_n'].sum()
    H = data['hispanic_n'].sum()
    W = data['white_n'].sum()
    BH = B + H  # Black-Hispanic combined
    AW = A + W  # Asian-White combined
    N = data['total_enrollment'].sum()

    # Calculate "Non-Group" totals
    non_A = N - A
    non_W = N - W

    # Calculate proportions for each group and their complements
    data['p_a'] = data['asian_n'] / A
    data['p_non_a'] = (data['total_enrollment'] - data['asian_n']) / non_A

    data['p_w'] = data['white_n'] / W
    data['p_non_w'] = (data['total_enrollment'] - data['white_n']) / non_W

    data['p_bh'] = (data['black_n'] + data['hispanic_n']) / BH
    data['p_aw'] = (data['asian_n'] + data['white_n']) / AW

    # Calculate Dissimilarity Index for each comparison
    D_asian = (data['p_a'] - data['p_non_a']).abs().sum() / 2
    D_non_white = (data['p_w'] - data['p_non_w']).abs().sum() / 2
    D_black_hispanic = (data['p_bh'] - data['p_aw']).abs().sum() / 2
    return D_asian, D_non_white, D_black_hispanic



In [85]:
# calculate dissimilarity for each district in NYCPS
t = ["NYCPS", *calc_D(df.copy())]
for d in df.district.unique():
    t.extend([d, *calc_D(df[df.district == d].copy())])

diss = pd.DataFrame(columns=["district", "asian_d", "non_white_d", "black_hispanic_d"], data=np.array(t).reshape(-1, 4))
diss.sort_values(by="black_hispanic_d", ascending=False)

,district,asian_d,non_white_d,black_hispanic_d
13,13,0.6701569130356794,0.3926845104144236,0.6259420571746532
0,NYCPS,0.5476491261340923,0.5684714632157319,0.6031986212637963
10,10,0.5907860695944609,0.5411240613454735,0.6017576168782914
5,5,0.537816246940532,0.5479882146976124,0.5892834830055006
35,84,0.5359997980574033,0.6242665109017775,0.5862933622087521
2,2,0.46485546447337855,0.47979741109412494,0.5764986997345212
14,14,0.473308129529294,0.554797063631935,0.5725736362351402
3,3,0.3967366680036763,0.39130125972911395,0.5109917549213527
1,1,0.47670364830050693,0.4436379328692973,0.4972303481573455
29,29,0.5021218686473634,0.27975918409029615,0.49705302846540445


In [86]:
# join them with district aggregates
agg = {"total_enrollment": "sum", 
       "poverty_pct": "mean", 
       "ell_pct": "mean", 
       "asian_pct": "mean", 
       "black_pct": "mean", 
       "hispanic_pct": "mean", 
       "white_pct": "mean"}

x = pd.DataFrame([df.agg(agg)])
x["district"] = "NYCPS"
x.reset_index()


districts = df.groupby("district").agg(agg).reset_index()
districts = pd.concat([x, districts])
districts.district = districts.district.astype(str)
diss = diss.merge(districts, on="district")
diss

,district,asian_d,non_white_d,black_hispanic_d,total_enrollment,poverty_pct,ell_pct,asian_pct,black_pct,hispanic_pct,white_pct
0,NYCPS,0.5476491261340923,0.5684714632157319,0.6031986212637963,987787.0,0.775939,0.151943,0.114237,0.291878,0.442471,0.112103
1,1,0.47670364830050693,0.4436379328692973,0.4972303481573455,9470.0,0.731559,0.081927,0.125438,0.191427,0.499924,0.136576
2,2,0.46485546447337855,0.47979741109412494,0.5764986997345212,55347.0,0.568989,0.106243,0.169502,0.154687,0.370497,0.231004
3,3,0.3967366680036763,0.39130125972911395,0.5109917549213527,18867.0,0.599070,0.072855,0.068038,0.239747,0.391452,0.236945
4,4,0.49637566917018594,0.33879329827121724,0.4550124671969981,10899.0,0.845965,0.104726,0.062872,0.259084,0.584736,0.054014
5,5,0.537816246940532,0.5479882146976124,0.5892834830055006,8101.0,0.860052,0.086892,0.028358,0.484418,0.389927,0.056342
6,6,0.37217620763671455,0.5600741551903741,0.49344316537905136,17152.0,0.845703,0.297445,0.011439,0.064335,0.834968,0.061525
7,7,0.33987748483054864,0.26812777728730763,0.22663946683162334,14695.0,0.925867,0.161753,0.009764,0.269460,0.677782,0.015568
8,8,0.4136142029486914,0.3791259909239475,0.4147675737588198,22443.0,0.860856,0.153159,0.066011,0.211702,0.645034,0.045114
9,9,0.41448425289130525,0.2969457601732469,0.3136330154447229,25367.0,0.933797,0.236756,0.010343,0.267843,0.687556,0.013379


In [88]:
corr = diss[[c for c in diss.columns if c != "district"]].corr()
corr.style.background_gradient(cmap='coolwarm')

,asian_d,non_white_d,black_hispanic_d,total_enrollment,poverty_pct,ell_pct,asian_pct,black_pct,hispanic_pct,white_pct
asian_d,1.000000,0.408983,0.779081,0.251140,0.124418,-0.162717,-0.364475,0.097450,0.261782,-0.250515
non_white_d,0.408983,1.000000,0.697244,0.298534,-0.148243,0.001588,-0.130865,-0.150213,0.181205,0.088056
black_hispanic_d,0.779081,0.697244,1.000000,0.312012,-0.273060,-0.170867,-0.164435,-0.117724,0.124617,0.179631
total_enrollment,0.251140,0.298534,0.312012,1.000000,-0.038930,0.007570,0.021950,-0.034773,0.002888,0.039033
poverty_pct,0.124418,-0.148243,-0.273060,-0.038930,1.000000,0.292028,-0.596979,0.335699,0.452146,-0.719482
ell_pct,-0.162717,0.001588,-0.170867,0.007570,0.292028,1.000000,0.117070,-0.609645,0.615337,-0.011362
asian_pct,-0.364475,-0.130865,-0.164435,0.021950,-0.596979,0.117070,1.000000,-0.492510,-0.394139,0.453713
black_pct,0.097450,-0.150213,-0.117724,-0.034773,0.335699,-0.609645,-0.492510,1.000000,-0.505793,-0.465529
hispanic_pct,0.261782,0.181205,0.124617,0.002888,0.452146,0.615337,-0.394139,-0.505793,1.000000,-0.326394
white_pct,-0.250515,0.088056,0.179631,0.039033,-0.719482,-0.011362,0.453713,-0.465529,-0.326394,1.000000
